<a href="https://colab.research.google.com/github/park1NG/winterTIL/blob/main/TeamProject_MalwareDetection_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 드라이브 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

In [ ]:
cd '/content/drive/Shareddrives/BigDataSecurity'

In [ ]:
ls

## 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.utils.data
import os
import time
# 전처리
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt

In [ ]:
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True # prevent truncate error

## 데이터셋 불러오기


In [ ]:
image_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),        
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

In [ ]:
malware_list = {'Adposhel': 0,
 'Allaple.A': 0,
 'Amonetize': 0,
 'Autorun': 0,
 'Benign Files': 1,
 'BrowseFox': 0,
 'Dinwod': 0,
 'InstallCore': 0,
 'MultiPlug': 0,
 'VBA': 0,
 'Vilsel': 0}

In [ ]:
def label_transform(label):
    # 예시: 클래스 레이블을 원하는 형태로 변환
    if label != 4:
      return 0
    else:
      return 1

In [ ]:
# import shutil

# train_folder = './Project/Train'
# validation_folder = './Project/Validation'

# ipynb_checkpoint_train = os.path.join(train_folder, '.ipynb_checkpoints')
# ipynb_checkpoint_validation = os.path.join(validation_folder, '.ipynb_checkpoints')

# if os.path.exists(ipynb_checkpoint_train):
#     shutil.rmtree(ipynb_checkpoint_train)

# if os.path.exists(ipynb_checkpoint_validation):
#     shutil.rmtree(ipynb_checkpoint_validation)

In [ ]:
datasets.ImageFolder(root = './Project/Train')

In [ ]:
train_data = datasets.ImageFolder(root = './Project/Train', 
                                  transform = image_transforms['train'], target_transform = label_transform)

test_data = datasets.ImageFolder(root = './Project/Validation', 
                                 transform = image_transforms['test'], target_transform = label_transform)

In [ ]:
val_size = int(0.5 * len(test_data))
test_size = int(0.5 * len(test_data))

valid_data, test_data = torch.utils.data.random_split(test_data, [val_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True) # make train loader
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=128, shuffle=False) # make test loader
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False) # make test loader

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
train_data.class_to_idx  = malware_list # class name

In [ ]:
classes = train_data.class_to_idx
classes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
# print(dataiter.next())
images, labels = next(dataiter)

batch_size = 16

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print()

labels = labels.tolist()
print(' '.join(f'{list(classes.keys())[list(classes.values()).index(j)]}' for j in labels))

## 전처리

In [ ]:
enhanced_data = []  # List to store enhanced images
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Laplacian Filtering (Edge 강화)

In [ ]:
if len(enhanced_data) == 0:
    # enhanced_data가 비어 있을 경우 train_loader를 사용하여 전처리
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
        enhanced_images = []
        # 라플라시안 필터 적용
        for image in images:
            # 라플라시안 필터 적용
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            edge = cv2.Laplacian(image, -1)
            filtered_image = np.hstack((image, edge))
            enhanced_images.append(filtered_image)
        enhanced_images = np.array(enhanced_images)
        enhanced_data.append((enhanced_images.squeeze(), labels))

        # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
        enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])
else:
    # enhanced_data가 비어 있지 않을 경우 enhanced_loader를 사용하여 전처리
    filtered_data = []  # 필터링된 데이터를 임시로 저장할 리스트를 생성합니다.
    for data in enhanced_loader:
        images, labels = data[0].to(device), data[1].to(device)
        enhanced_images = []
        # 라플라시안 필터 적용
        for image in images:
            # 라플라시안 필터 적용
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            edge = cv2.Laplacian(image, -1)
            filtered_image = np.hstack((image, edge))
            enhanced_images.append(filtered_image)
        enhanced_images = np.array(enhanced_images)
        enhanced_data.append((enhanced_images.squeeze(), labels))

        # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
        enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])


In [ ]:
# Create enhanced DataLoader
enhanced_loader = DataLoader(enhanced_data, batch_size=128, shuffle=True)

###Bilateral Filtering (노이즈 제거)

In [ ]:
if len(enhanced_data) == 0:
    # enhanced_data가 비어 있을 경우 train_loader를 사용하여 전처리
    for data in train_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (24, 224))  # 이미지 크기 조정
            filtered_image = cv2.bilateralFilter(image, -1, 10, 5)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((filtered_image.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])
else:
     # enhanced_data가 비어 있지 않을 경우 enhanced_loader를 사용하여 전처리
    filtered_data = []  # 필터링된 데이터를 임시로 저장할 리스트를 생성합니다.
    for data in enhanced_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            filtered_image = cv2.bilateralFilter(image, -1, 10, 5)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((enhanced_images.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])

In [ ]:
# Create enhanced DataLoader
enhanced_loader = DataLoader(enhanced_data, batch_size=128, shuffle=True)

### Contrast 조정 ( CLAHE )

In [ ]:
if len(enhanced_data) == 0:
    # enhanced_data가 비어 있을 경우 train_loader를 사용하여 전처리
    for data in train_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(4,4))
            filtered_images = clahe.apply(image)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((enhanced_images.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])
else:
     # enhanced_data가 비어 있지 않을 경우 enhanced_loader를 사용하여 전처리
    filtered_data = []  # 필터링된 데이터를 임시로 저장할 리스트를 생성합니다.
    for data in enhanced_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            image = image.astype(np.float32)  # 데이터 타입을 float32로 변환
            clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(4,4))
            filtered_images = clahe.apply(image)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((enhanced_images.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])

In [ ]:
# Create enhanced DataLoader
enhanced_loader = DataLoader(enhanced_data, batch_size=128, shuffle=True)

###  영상 압축( Wavelet Transform )

In [ ]:
import pywt
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Wavelet transform
wavelet = 'db5'  # Daubechies family
level = 2  # Number of decomposition levels
coeffs = pywt.wavedec2(image, wavelet, level=level)

# 재구성
reconstructed_image = pywt.waverec2(coeffs, wavelet)

In [ ]:
if len(enhanced_data) == 0:
    # enhanced_data가 비어 있을 경우 train_loader를 사용하여 전처리
    for data in train_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            coeffs = pywt.wavedec2(image, wavelet, level=level)
            filtered_image = pywt.waverec2(coeffs, wavelet)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((filtered_image.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])
else:
     # enhanced_data가 비어 있지 않을 경우 enhanced_loader를 사용하여 전처리
    filtered_data = []  # 필터링된 데이터를 임시로 저장할 리스트를 생성합니다.
    for data in enhanced_loader:
      images, labels = data[0].to(device), data[1].to(device)
      enhanced_images = []
      for image in images:
            image = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
            coeffs = pywt.wavedec2(image, wavelet, level=level)
            filtered_image = pywt.waverec2(coeffs, wavelet)
            enhanced_images.append(filtered_image)
      enhanced_images = np.array(enhanced_images)
      enhanced_data.append((enhanced_images.squeeze(), labels))

      # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
      enhanced_data.extend([(enhanced_image.squeeze(), label) for enhanced_image, label in zip(enhanced_images, labels)])

In [ ]:
# Create enhanced DataLoader
enhanced_loader = DataLoader(enhanced_data, batch_size=128, shuffle=True)

### HOG

In [ ]:
from skimage.feature import hog


In [ ]:
def load_hog_features(dir):
    features = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            print("Processing directory:", subdir_path)
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                img_path = os.path.join(subdir_path, filename)
                img = cv2.imread(img_path,0)
                features.append(hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False))
    features.shape()
    return np.array(features)

In [ ]:
dir = "Train/"
#HOG_descriptors = load_hog_features(dir)

HOG_descriptors.shape
GIST_descriptors.shape

###GIST

In [ ]:
pip install scikit-image

In [ ]:
import cv2
import numpy as np
from skimage.util import view_as_blocks

In [ ]:
# Gabor 필터 생성 함수
def create_filters(scales, orientations):
    filters = []
    for scale in range(scales[0], scales[1] + 1):
        for orientation in np.arange(0, np.pi, np.pi / orientations):
            filt_real = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0, ktype=cv2.CV_32F)
            filt_imag = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0.5 * np.pi, ktype=cv2.CV_32F)
            filt = filt_real + filt_imag
            filt /= 2.0 * np.pi * scale * scale
            filters.append(filt)
    return filters

# GIST 디스크립터 계산 함수
def gist_descriptor_single_channel(image, scales=(8, 8), orientations=8, blocks=(4, 4)):    # Gabor 필터 생성
    filters = create_filters(scales, orientations)
    
    # 이미지 크기와 블록 크기 계산
    height, width = image.shape[:2]
    block_size = height // blocks[0], width // blocks[1]

    padding_size = blocks[0] * block_size[0] - height, blocks[1] * block_size[1] - width
    
    # 이미지 패딩 (필요한 경우)
    if padding_size != (0, 0):
        image = cv2.copyMakeBorder(image, 0, padding_size[0], 0, padding_size[1], cv2.BORDER_CONSTANT, value=0)
    
    # 이미지를 블록으로 분할
    block_shape = (block_size[0], block_size[1])
    blocks = view_as_blocks(image, block_shape=(block_size[0], block_size[1])).reshape(-1, *block_size, order='F')
    
    # 각 블록의 GIST 특성 추출
    features = []
    for block in blocks:
        feats = []
        for scale in filters:
            for filt in scale:
                filtered = cv2.filter2D(block, cv2.CV_64F, filt)
                feats.append(filtered.mean())
        features.append(feats)
    
    # 전체 GIST 디스크립터로 결합
    return np.concatenate(features)


def gist_descriptor(image, scales=(8, 8), orientations=8, blocks=(4, 4)):
    if len(image.shape) == 3:
    # 각 채널에 대해 GIST 디스크립터 계산
        descriptors = [gist_descriptor_single_channel(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), scales, orientations, blocks)]            # 전체 GIST 디스크립터로 결합
        return np.concatenate(descriptors)
    else:
    # 단일 채널 이미지의 경우 GIST 디스크립터를 한 번만 계산
        return gist_descriptor_single_channel(image, scales, orientations, blocks)

In [ ]:
cd 

In [ ]:
# 각 악성코드 이미지 폴더에서 350개의 이미지에 대한 gist descriptor를 계산하여 반환
def get_gist_descriptors(root_dir):
    descriptors = []
    for subdir in sorted(os.listdir(root_dir)):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path):
            print("Processing directory:", subdir_path)
            for i, filename in enumerate(os.listdir(subdir_path)):
                # 파일 경로 생성
                filepath = os.path.join(subdir_path, filename)
                # 이미지 로드
                image = cv2.imread(filepath)
                # 이미지에 대한 GIST 디스크립터 계산
                descriptor = gist_descriptor(image)
                descriptors.append(descriptor)
                
                if i % 10 == 9:
                    print("\tProcessed", i + 1, "images")
    return np.array(descriptors)


In [ ]:
root_dir = "Train/"
# 각 폴더에서 350개의 이미지에 대한 gist descriptor 계산
descriptors = get_gist_descriptors(root_dir)
print(len(descriptors))
print('GIST Descriptor Shape:', descriptors.shape)

In [ ]:
GIST_descriptors = descriptors

###SIFT

In [ ]:
sift_features = []

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sift= cv2.xfeatures2d.SIFT_create()

In [ ]:
import numpy as np

if len(enhanced_data) == 0:
    # enhanced_data가 비어 있을 경우 train_loader를 사용하여 전처리
    for data in train_loader:
        features = []
        images, labels = data[0].to(device), data[1].to(device)
        for image, label in zip(images, labels):
            image_np = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)  # RGB 이미지를 그레이스케일 이미지로 변환
            gray = gray.astype(np.uint8)  # 데이터 타입을 np.uint8로 변환
            kp, des = sift.detectAndCompute(gray, None)
            if des is None:
                continue  # SIFT 특징을 검출하지 못한 경우 다음 이미지로 넘어감
            sift_feature = des
            features.append((sift_feature.squeeze(), label))
        features = np.array(features)
        sift_features.append((features.squeeze(), labels))

        sift_features.extend([(sift_feature.squeeze(), label) for sift_feature, label in zip(features, labels)])
else:
    # enhanced_data가 비어 있지 않을 경우 enhanced_loader를 사용하여 전처리
    for data in enhanced_loader:
        features = []
        images, labels = data[0].to(device), data[1].to(device)
        for image, label in zip(images, labels):
            image_np = image.permute(1, 2, 0).cpu().numpy()  # PyTorch tensor를 NumPy 배열로 변환
            gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)  # RGB 이미지를 그레이스케일 이미지로 변환
            gray = gray.astype(np.uint8)  # 데이터 타입을 np.uint8로 변환
            kp, des = sift.detectAndCompute(gray, None)
            if des is None:
                continue  # SIFT 특징을 검출하지 못한 경우 다음 이미지로 넘어감
            sift_feature = des
            features.append((sift_feature.squeeze(), label))
        features = np.array(features)
        sift_features.append((features.squeeze(), labels))

        # enhanced_data에 필터링된 이미지와 레이블을 추가합니다.
        sift_features.extend([(sift_feature.squeeze(), label) for sift_feature, label in zip(features, labels)])


##EDA

### 모델 불러오기

In [ ]:
from sklearn.manifold import TSNE # sklearn 사용하면 easy !! 
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

시각화에 사용할 모델 불러오기

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features # fc의 입력 노드 수를 산출한다. 512개
model.fc = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체, CIFAR10,,
model = model.to(device)

In [ ]:
# classifier 들어가기 직전에 값을 뽑아낼 것임
# (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
print(model)

###T-SNE

기본 데이터 T-SNE

In [ ]:
actual = []
deep_features = []

# Define a color map for each class
color_map = plt.cm.get_cmap('tab10', len(malware_list))

model.eval() # Set the model to evaluation mode
with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features = model(images)

        deep_features += features.cpu().numpy().tolist()
        actual += labels.cpu().numpy().tolist()

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
cluster = np.array(tsne.fit_transform(np.array(deep_features)))
actual = np.array(actual)

plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(actual == malware_type)
    plt.scatter(cluster[idx, 0], cluster[idx, 1], marker='.', label=label, color=color_map(i))

plt.legend()
plt.show()

전처리한 데이터 T-SNE

In [ ]:
actual = []
deep_features = []

model.eval() # Set the model to evaluation mode
with torch.no_grad():
    for data in enhanced_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features = model(images)

        deep_features += features.cpu().numpy().tolist()
        actual += labels.cpu().numpy().tolist()

tsne = TSNE(n_components=2, random_state=0)
cluster = np.array(tsne.fit_transform(np.array(deep_features)))
actual = np.array(actual)

plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(actual == malware_type)
    plt.scatter(cluster[idx, 0], cluster[idx, 1], marker='.', label=label)

plt.legend()
plt.show()

SIFT Feature T-SNE

In [ ]:
# Separate features and labels
features = np.array([f[0] for f in sift_features])
labels = np.array([f[1] for f in sift_features])

# T-SNE를 사용하여 features를 2차원으로 축소합니다.
tsne = TSNE(n_components=2, random_state=0, perplexity=5)
cluster = np.array(tsne.fit_transform(features))

# 시각화를 위해 산점도를 그립니다.
plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(labels == malware_type)
    plt.scatter(cluster[idx, 0], cluster[idx, 1], marker='.', label=label)

plt.legend()
plt.show()


### UMAP

기본 데이터 UMAP

In [ ]:
actual = []
deep_features = []

model.eval() # Set the model to evaluation mode
with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features = model(images)

        deep_features += features.cpu().numpy().tolist()
        actual += labels.cpu().numpy().tolist()

umap_emb = umap.UMAP(n_components=2, random_state=0)
embedded = umap_emb.fit_transform(np.array(deep_features))
actual = np.array(actual)

plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(actual == malware_type)
    plt.scatter(embedded[idx, 0], embedded[idx, 1], marker='.', label=label)

plt.legend()
plt.show()

전처리한 데이터 UMAP

In [ ]:
actual = []
deep_features = []

model.eval() # Set the model to evaluation mode
with torch.no_grad():
    for data in enhanced_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features = model(images)

        deep_features += features.cpu().numpy().tolist()
        actual += labels.cpu().numpy().tolist()

umap_emb = umap.UMAP(n_components=2, random_state=0)
embedded = umap_emb.fit_transform(np.array(deep_features))
actual = np.array(actual)

plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(actual == malware_type)
    plt.scatter(embedded[idx, 0], embedded[idx, 1], marker='.', label=label)

plt.legend()
plt.show()

SIFT UMAP

In [ ]:
sift_features = np.array(sift_features)  # sift_features를 numpy 배열로 변환

# UMAP을 사용하여 차원 축소
reducer = umap.UMAP(n_components=2)
embedding = reducer.fit_transform(sift_features)

# 시각화
plt.figure(figsize=(10, 10))
for label, malware_type in malware_list.items():
    idx = np.where(actual == malware_type)
    plt.scatter(embedding[idx, 0], embedding[idx, 1], marker='.', label=label)

plt.legend()
plt.show()


## 모델 구성

## Machine Learning

with GIST

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier


In [ ]:

# 데이터와 레이블 설정
X = descriptors

# 레이블 기록
labels_dict = classes
train_dir = "Train/"  # 이 경로에 train 데이터가 저장
val_dir = "Validation/"  # 이 경로에 test 데이터가 저장

# GIST_descriptors 및 레이블 불러오기
X_train = X
X_test = get_gist_descriptors(val_dir)

# 레이블 설정
labels_dict = classes

# 각 이미지에 맞는 레이블 생성
def load_labels(dir):
    y = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                y.append(labels_dict[subdir])
    return np.array(y)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Linear SVM': LinearSVC(),
    'SMO': SVC(kernel='rbf'), # SMO는 일반적으로 서포트 벡터 머신 (SVM)이 rbf 커널을 사용
    'J48': DecisionTreeClassifier() # J48는 scikit-learn에서 Decision Tree에 해당
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, y_train)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(y_test, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(y_test, y_pred))
        else:
            print(score_name, score_func(y_test, y_pred, average='weighted'))


with GIST+HOG

In [ ]:
hog_gist_concatenated = np.concatenate((HOG_descriptors, GIST_descriptors), axis=1)
hog_gist_concatenated.shape

In [ ]:

# 데이터와 레이블 설정
X = hog_gist_concatenated

# 레이블 기록
labels_dict = classes
train_dir = "Train/"  # 이 경로에 train 데이터가 저장
val_dir = "Validation/"  # 이 경로에 test 데이터가 저장

# GIST_descriptors 및 레이블 불러오기
X_train = X
X_test = get_gist_descriptors(val_dir)

# 레이블 설정
labels_dict = classes

# 각 이미지에 맞는 레이블 생성
def load_labels(dir):
    y = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                y.append(labels_dict[subdir])
    return np.array(y)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Length of X:", len(X))
print("Length of y:", len(y))

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Linear SVM': LinearSVC(),
    'SMO': SVC(kernel='rbf'), # SMO는 일반적으로 서포트 벡터 머신 (SVM)이 rbf 커널을 사용
    'J48': DecisionTreeClassifier() # J48는 scikit-learn에서 Decision Tree에 해당
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, y_train)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(y_test, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(y_test, y_pred))
        else:
            print(score_name, score_func(y_test, y_pred, average='weighted'))


## Deep Learning

### Model List

#### Model-DNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

하이퍼파라미터 설정

In [ ]:
batch_size = 128
learning_rate = 0.001
patience = 5

모델 정의

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        ############### Conv2d, MaxPool2d, Linear 함수에 들어갈 파라미터를 채우세요 ##############
        self.conv1 = nn.Conv2d(3, 10, 3) # in_channel, out_channel, kernel size
        self.pool = nn.MaxPool2d(3, 2) # kernel_size, stride
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(56180, 160) # in_features, out_features
        self.fc2 = nn.Linear(160, 120)
        self.fc3 = nn.Linear(120, 26)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.2)
        ###########################################################################################


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DNN().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)
print(output.size())

summary(model, (3, 224, 224))

학습 및 평가 함수 정의

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for batch_x, batch_y in iterator:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == batch_y).float().mean()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for batch_x, batch_y in iterator:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            output = model(batch_x)
            loss = criterion(output, batch_y)
            acc = (output.argmax(dim=1) == batch_y).float().mean()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Early Stopping 클래스 정의

In [ ]:
class EarlyStopping:
    def __init__(self, patience):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss > self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [ ]:
# EarlyStopping 객체 생성
num_epochs = 100
early_stopping = EarlyStopping(patience=10)

학습

In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)

    print(f"Epoch: {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}")

    if early_stopping.early_stop:
        print("Early stopping")
        break

학습 평가

In [ ]:
# 테스트 데이터로 최종 평가
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

#### Model-CNN basic

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        ############### Conv2d, MaxPool2d, Linear 함수에 들어갈 파라미터를 채우세요 ##############
        self.conv1 = nn.Conv2d(3, 10, 3) # in_channel, out_channel, kernel size
        self.pool = nn.MaxPool2d(3, 2) # kernel_size, stride
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(56180, 160) # in_features, out_features
        self.fc2 = nn.Linear(160, 120)
        self.fc3 = nn.Linear(120, 26)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.2)
        ###########################################################################################


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Use GPU if it's available # colab 런타임 유형 변경에서 GPU 선택할 것
model = Net().to(device) # define the network

In [ ]:
x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)
print(output.size())

summary(model, (3, 224, 224))

#### Model-ResNet50

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, config, output_dim):
        super().__init__()
                
        block, n_blocks, channels = config
        self.in_channels = channels[0]
            
        assert len(n_blocks) == len(channels) == 4
        
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size = 7, stride = 2, padding = 3, bias = False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self.get_model_layer(block, n_blocks[0], channels[0])
        self.layer2 = self.get_model_layer(block, n_blocks[1], channels[1], stride = 2)
        self.layer3 = self.get_model_layer(block, n_blocks[2], channels[2], stride = 2)
        self.layer4 = self.get_model_layer(block, n_blocks[3], channels[3], stride = 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.in_channels, output_dim)
        
    def get_model_layer(self, block, n_blocks, channels, stride = 1):
    
        layers = []
        
        if self.in_channels != block.expansion * channels:
            downsample = True
        else:
            downsample = False
        
        layers.append(block(self.in_channels, channels, stride, downsample))
        
        for i in range(1, n_blocks):
            layers.append(block(block.expansion * channels, channels))

        self.in_channels = block.expansion * channels
            
        return nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.fc(h)
        
        return x, h

In [ ]:
class BasicBlock(nn.Module):
    
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride = 1, downsample = False):
        super().__init__()
                
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, 
                               stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, 
                               stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU(inplace = True)
        
        if downsample:
            conv = nn.Conv2d(in_channels, out_channels, kernel_size = 1, 
                             stride = stride, bias = False)
            bn = nn.BatchNorm2d(out_channels)
            downsample = nn.Sequential(conv, bn)
        else:
            downsample = None
        
        self.downsample = downsample
        
    def forward(self, x):
        
        i = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        if self.downsample is not None:
            i = self.downsample(i)
                        
        x += i
        x = self.relu(x)
        
        return x

In [ ]:
class Bottleneck(nn.Module):
    
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride = 1, downsample = False):
        super().__init__()
    
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 1, 
                               stride = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, 
                               stride = stride, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, self.expansion * out_channels, kernel_size = 1,
                               stride = 1, bias = False)
        self.bn3 = nn.BatchNorm2d(self.expansion * out_channels)
        
        self.relu = nn.ReLU(inplace = True)
        
        if downsample:
            conv = nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size = 1, 
                             stride = stride, bias = False)
            bn = nn.BatchNorm2d(self.expansion * out_channels)
            downsample = nn.Sequential(conv, bn)
        else:
            downsample = None
            
        self.downsample = downsample
        
    def forward(self, x):
        
        i = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
                
        if self.downsample is not None:
            i = self.downsample(i)
            
        x += i
        x = self.relu(x)
    
        return x

##### model_info

In [ ]:
from collections import namedtuple
Model_Info = namedtuple('Model_Info', ['block', 'n_blocks', 'channels'])

Model_config = Model_Info(block = Bottleneck,
                               n_blocks = [3, 4, 6, 3],
                               channels = [64, 128, 256, 512])

In [ ]:
import torchvision.models as models
pretrained_model = models.resnet50(pretrained = True)

# Fine Tuning
IN_FEATURES = pretrained_model.fc.in_features 
OUTPUT_DIM = 10

fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
pretrained_model.fc = fc
model = CustomModel(Model_config, OUTPUT_DIM)

In [ ]:
# Model Load
model.load_state_dict(pretrained_model.state_dict())

In [ ]:
# Count trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

### LR_finder

In [ ]:
START_LR = 1e-7

optimizer = optim.Adam(model.parameters(), lr=START_LR)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from torchsummary import summary
from torchvision import models

x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)

summary(model, (3, 224, 224))

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler

class LRFinder:
    def __init__(self, model, optimizer, criterion, device):
        
        self.optimizer = optimizer
        self.model = model
        self.criterion = criterion
        self.device = device
        
        torch.save(model.state_dict(), 'init_params.pt')

    def range_test(self, iterator, end_lr = 10, num_iter = 100, 
                   smooth_f = 0.05, diverge_th = 5):
        
        lrs = []
        losses = []
        best_loss = float('inf')

        lr_scheduler = ExponentialLR(self.optimizer, end_lr, num_iter)
        
        iterator = IteratorWrapper(iterator)
        
        for iteration in range(num_iter):

            loss = self._train_batch(iterator)

            #update lr
            lr_scheduler.step()
            
            lrs.append(lr_scheduler.get_lr()[0])

            if iteration > 0:
                loss = smooth_f * loss + (1 - smooth_f) * losses[-1]
                
            if loss < best_loss:
                best_loss = loss

            losses.append(loss)
            
            if loss > diverge_th * best_loss:
                print("Stopping early, the loss has diverged")
                break
                       
        #reset model to initial parameters
        model.load_state_dict(torch.load('init_params.pt'))
                    
        return lrs, losses

    def _train_batch(self, iterator):
        
        self.model.train()
        
        self.optimizer.zero_grad()
        
        x, y = iterator.get_batch()
        
        x = x.to(self.device)
        y = y.to(self.device)
        
        y_pred, _ = self.model(x)
                
        loss = self.criterion(y_pred, y)
        
        loss.backward()
        
        self.optimizer.step()
        
        return loss.item()

class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch + 1
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in self.base_lrs]

class IteratorWrapper:
    def __init__(self, iterator):
        self.iterator = iterator
        self._iterator = iter(iterator)

    def __next__(self):
        try:
            inputs, labels = next(self._iterator)
        except StopIteration:
            self._iterator = iter(self.iterator)
            inputs, labels, *_ = next(self._iterator)

        return inputs, labels

    def get_batch(self):
        return next(self)

In [ ]:
END_LR = 10
NUM_ITER = 30

lr_finder = LRFinder(model, optimizer, criterion, device)
lrs, losses = lr_finder.range_test(train_loader, END_LR, NUM_ITER)

In [ ]:
def plot_lr_finder(lrs, losses, skip_start = 10, skip_end = 30):
    
    if skip_end == 0:
        lrs = lrs[skip_start:]
        losses = losses[skip_start:]
    else:
        lrs = lrs[skip_start:-skip_end]
        losses = losses[skip_start:-skip_end]
    
    fig = plt.figure(figsize = (16,8))
    ax = fig.add_subplot(1,1,1)
    ax.plot(lrs, losses)
    ax.set_xscale('log')
    ax.set_xlabel('Learning rate')
    ax.set_ylabel('Loss')
    ax.grid(True, 'both', 'x')
    plt.show()

In [ ]:
FOUND_LR = 1e-3

params = [
          {'params': model.conv1.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.bn1.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.layer1.parameters(), 'lr': FOUND_LR / 8},
          {'params': model.layer2.parameters(), 'lr': FOUND_LR / 6},
          {'params': model.layer3.parameters(), 'lr': FOUND_LR / 4},
          {'params': model.layer4.parameters(), 'lr': FOUND_LR / 2},
          {'params': model.fc.parameters()}
         ]


optimizer = optim.Adam(params, lr = FOUND_LR, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [ ]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='./net_pretrained.pth'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: './net_pretrained.pth'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## 모델 학습

In [ ]:
def calculate_topk_accuracy(y_pred, y, k = 5):
    with torch.no_grad():
        batch_size = y.shape[0]
        _, top_pred = y_pred.topk(k, 1)
        top_pred = top_pred.t()
        correct = top_pred.eq(y.view(1, -1).expand_as(top_pred))
        correct_1 = correct[:1].reshape(-1).float().sum(0, keepdim = True)
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim = True)
        acc_1 = correct_1 / batch_size
        acc_k = correct_k / batch_size
    return acc_1, acc_k

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc_1 += acc_1.item()
        epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.eval()
    
    with torch.no_grad():

        for (x, y) in iterator:

           x = x.to(device)
           y = y.to(device)

           y_pred, _ = model(x)

           loss = criterion(y_pred, y)

           acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)

           epoch_loss += loss.item()
           epoch_acc_1 += acc_1.item()
           epoch_acc_5 += acc_5.item()

          
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')
result_list = []
lr_list = []

patience = 5

early_stopping = EarlyStopping(patience = patience, verbose = True)

for epoch in range(100):
    
    start_time = time.monotonic()
    
    train_loss, train_acc_1, train_acc_5 = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc_1, valid_acc_5 = evaluate(model, valid_loader, criterion, device)
      
    early_stopping(valid_loss, model)
    lr_list.append(optimizer.param_groups[0]["lr"]) 

    # patience 동안 val_loss가 감소하지 않으면 조기 종료
    if early_stopping.early_stop:
      print("Early stopping")
      break

    # val_loss 감소하면 best model 불러오기 
    model.load_state_dict(torch.load('./net_pretrained.pth'))

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc_1: {train_acc_1*100:6.2f}% | Train Acc_5: {train_acc_5*100:6.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc_1: {valid_acc_1*100:6.2f}% | Train Acc_5: {train_acc_5*100:6.2f}%')

    result = {
    'EPOCH': epoch,
    'Train Loss': train_loss,
    'Train acc_1': train_acc_1,
    'Train acc_5': train_acc_5,
    'Valid Loss': valid_loss,
    'Valid acc_1': valid_acc_1,
    'Valid acc_5': valid_acc_5}
  
    result_list.append(result)
  
result_df = pd.DataFrame(result_list)

## 모델 평가

In [ ]:
# Loss 및 Acc 변화 그래프
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

axes[0].plot(result_df['EPOCH'], result_df['Train Loss'], label='Train Loss')
axes[0].plot(result_df['EPOCH'], result_df['Valid Loss'], label='Valid Loss')
axes[0].legend()
axes[0].set_title('Loss')

axes[1].plot(result_df['EPOCH'], result_df['Train acc_1'], label='Train acc_1')
axes[1].plot(result_df['EPOCH'], result_df['Valid acc_1'], label='Valid acc_1')
axes[1].legend()
axes[1].set_title('ACC_1')

axes[2].plot(result_df['EPOCH'], result_df['Train acc_5'], label='Train acc_5')
axes[2].plot(result_df['EPOCH'], result_df['Valid acc_5'], label='Valid acc_5')
axes[2].legend()
axes[2].set_title('ACC_5')

plt.show()

In [ ]:
plt.plot(lr_list)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")

In [ ]:
model.load_state_dict(torch.load('./net_pretrained.pth'))

In [ ]:
test_loss, test_acc_1, test_acc_5 = evaluate(model, test_loader, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc @1: {test_acc_1*100:6.2f}% | Test Acc @5: {test_acc_5*100:6.2f}%')